# Step 11 - Improving the distance algorithm 

The distance algorithm is now working in general. This notebook intends to wrap it up and make it work faster if possible. Following problems are still not completely resolved:
- a step size of at least 2 ist still required to get results
- to avoid singular matrices, I had to round the value of the scalar field I am handing to the marching cubes method
  to 2 decimals.
      - not sure if that is robust for every possible case
      - increases the error that comes from choosing the closest point of the triangular mesh

Following ideas may speed up the calculation process:
In general:
- the triangular mesh for the basic point can be reused, as well as its prefactorization
- a limited amount of triangular meshes could be used (and reused)
- increase step size for forward simulation - this helps a LOT (20s for 50 points)
- parallelization ???

For Kriging:
- mix of eucl. dist and this function, as this becomes only important for distances, where layer curvature plays a role
- reduce number of points used 

Other things to do:
- add the factors for anisotropy
- check error tolerance and relevance
- rounding to reasonable decimal (0)

In [1]:
# These lines are necessary only if GemPy is not installed
import sys, os
sys.path.append("../../..")
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['MKL_THREADING_LAYER'] = 'GNU'

# Importing GemPy, which takes really long
import gempy as gp

# Importing auxiliary libraries
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import pandas as pd

from scipy import sparse
from scipy.sparse.linalg import splu

from skimage import measure
from scipy.spatial.distance import cdist

C:\Users\Jan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Importing the data from CSV-files and setting extent and resolution, example without faults
geo_data = gp.create_data([0,3000,0,200,0,2000],resolution=[120,4,80], 
                         path_o = "C:/Users/Jan/gempy/notebooks/input_data/tut_chapter3/tutorial_ch3_foliations", # importing orientation (foliation) data
                         path_i = "C:/Users/Jan/gempy/notebooks/input_data/tut_chapter3/tutorial_ch3_interfaces") # importing point-positional interface data

C:\Users\Jan\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [3]:
interp_data = gp.InterpolatorData(geo_data, u_grade=[1], output='geology', compile_theano=True,
                                  theano_optimizer='fast_compile')

interp_data_grad = gp.InterpolatorData(geo_data, u_grade=[1], output='gradients', compile_theano=True,
                                  theano_optimizer='fast_compile')

Compiling theano function...


WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


Compilation Done!
Level of Optimization:  fast_compile
Device:  cpu
Precision:  float32
Number of faults:  0
Compiling theano function...
Compilation Done!
Level of Optimization:  fast_compile
Device:  cpu
Precision:  float32
Number of faults:  0


In [4]:
lith_block, fault_block = gp.compute_model(interp_data)

C:\Users\Jan\Anaconda3\lib\site-packages\theano\tensor\subtensor.py:2320: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[0][inputs[2:]] = inputs[1]


In [5]:
def init_domain (geomodel, grid, formation):
        """
        Method to create a new pandas dataframe containing a grid for the SGS. Grid from complete geologic model is
        down to a certain formation of interest.
        Args:
            geomodel (numpy.ndarray): lithological block model created with gempy
            grid (gempy.data_management.GridClass): Grid created for geologic model
            formation (int): Number of formation to perform CoKriging on
        Returns:
            pandas.dataframe: Dataframe with all relevant data for grid, cut to one lith
        """
    
        # convert lith block values to int, thats what Miguel suggested
        lith_block_int = np.round(lith_block)
    
        # create the dataframe and populate with data
        d = {'X': grid.values[:,0], 'Y': grid.values[:,1], 'Z': grid.values[:,2], 'lith': lith_block_int[0], 'grad': lith_block[1]}
        df_cokr = pd.DataFrame(data=d)

        # cut down to wanted lithology and reset dataframne
        df_sgs_grid = df_cokr.loc[df_cokr['lith'] == formation]
        df_sgs_grid = df_sgs_grid.reset_index() # reset indicies
        del df_sgs_grid['index'] # reset indices

        return df_sgs_grid

In [8]:
def veclen(vectors):
    """ return L2 norm (vector length) along the last axis, for example to compute the length of an array of vectors """
    return np.sqrt(np.sum(vectors**2, axis=-1))

def normalized(vectors):
    """ normalize array of vectors along the last axis """
    return vectors / veclen(vectors)[..., np.newaxis]


def compute_mesh_laplacian(verts, tris):
    """
    computes a sparse matrix representing the discretized laplace-beltrami operator of the mesh
    given by n vertex positions ("verts") and a m triangles ("tris") 
    
    verts: (n, 3) array (float)
    tris: (m, 3) array (int) - indices into the verts array
    computes the conformal weights ("cotangent weights") for the mesh, ie:
    w_ij = - .5 * (cot \alpha + cot \beta)
    See:
        Olga Sorkine, "Laplacian Mesh Processing"
        and for theoretical comparison of different discretizations, see 
        Max Wardetzky et al., "Discrete Laplace operators: No free lunch"
    returns matrix L that computes the laplacian coordinates, e.g. L * x = delta
    """
    n = len(verts)
    W_ij = np.empty(0)
    I = np.empty(0, np.int32)
    J = np.empty(0, np.int32)
    for i1, i2, i3 in [(0, 1, 2), (1, 2, 0), (2, 0, 1)]: # for edge i2 --> i3 facing vertex i1
        vi1 = tris[:,i1] # vertex index of i1
        vi2 = tris[:,i2]
        vi3 = tris[:,i3]
        # vertex vi1 faces the edge between vi2--vi3
        # compute the angle at v1
        # add cotangent angle at v1 to opposite edge v2--v3
        # the cotangent weights are symmetric
        u = verts[vi2] - verts[vi1]
        v = verts[vi3] - verts[vi1]
        cotan = (u * v).sum(axis=1) / veclen(np.cross(u, v))
        W_ij = np.append(W_ij, 0.5 * cotan)
        I = np.append(I, vi2)
        J = np.append(J, vi3)
        W_ij = np.append(W_ij, 0.5 * cotan)
        I = np.append(I, vi3)
        J = np.append(J, vi2)
    L = sparse.csr_matrix((W_ij, (I, J)), shape=(n, n)) 
    
    # compute diagonal entries
    L = L - sparse.spdiags(L * np.ones(n), 0, n, n)
    L = L.tocsr()
    # area matrix
    e1 = verts[tris[:,1]] - verts[tris[:,0]]
    e2 = verts[tris[:,2]] - verts[tris[:,0]]
    n = np.cross(e1, e2)
    triangle_area = .5 * veclen(n)
    # compute per-vertex area
    vertex_area = np.zeros(len(verts))
    ta3 = triangle_area / 3
    for i in range(tris.shape[1]): # Jan: changed xrange to range
        bc = np.bincount(tris[:,i].astype(int), ta3)
        vertex_area[:len(bc)] += bc  
    VA = sparse.spdiags(vertex_area, 0, len(verts), len(verts))
    
    return L, VA


class GeodesicDistanceComputation(object):
    """ 
    Computation of geodesic distances on triangle meshes using the heat method from the impressive paper
        Geodesics in Heat: A New Approach to Computing Distance Based on Heat Flow
        Keenan Crane, Clarisse Weischedel, Max Wardetzky
        ACM Transactions on Graphics (SIGGRAPH 2013)
    Example usage:
        >>> compute_distance = GeodesicDistanceComputation(vertices, triangles)
        >>> distance_of_each_vertex_to_vertex_0 = compute_distance(0)
    """
    def __init__(self, verts, tris, m=10.0):
        self._verts = verts
        self._tris = tris
        # precompute some stuff needed later on
        e01 = verts[tris[:,1]] - verts[tris[:,0]]
        e12 = verts[tris[:,2]] - verts[tris[:,1]]
        e20 = verts[tris[:,0]] - verts[tris[:,2]]
        self._triangle_area = .5 * veclen(np.cross(e01, e12))
        unit_normal = normalized(np.cross(normalized(e01), normalized(e12)))
        self._unit_normal_cross_e01 = np.cross(unit_normal, e01)
        self._unit_normal_cross_e12 = np.cross(unit_normal, e12)
        self._unit_normal_cross_e20 = np.cross(unit_normal, e20)
        # parameters for heat method
        h = np.mean(list(map(veclen, [e01, e12, e20]))) # Jan: converted to list
        
        # Jan: m is constant optimized at 1, here 10 is used
        # Jan: h is mean distance between nodes/length of edges
        t = m * h ** 2
        
        # pre-factorize poisson systems
        Lc, A = compute_mesh_laplacian(verts, tris) 
        self._factored_AtLc = splu((A - t * Lc).tocsc()).solve
        self._factored_L = splu(Lc.tocsc()).solve
        
    def __call__(self, idx):
        """ 
        computes geodesic distances to all vertices in the mesh
        idx can be either an integer (single vertex index) or a list of vertex indices
        or an array of bools of length n (with n the number of vertices in the mesh) 
        """
        u0 = np.zeros(len(self._verts))
        u0[idx] = 1.0
        # heat method, step 1
        u = self._factored_AtLc(u0).ravel()
        # heat method step 2
        grad_u = 1 / (2 * self._triangle_area)[:,np.newaxis] * (
              self._unit_normal_cross_e01 * u[self._tris[:,2]][:,np.newaxis]
            + self._unit_normal_cross_e12 * u[self._tris[:,0]][:,np.newaxis]
            + self._unit_normal_cross_e20 * u[self._tris[:,1]][:,np.newaxis]
        )
        X = - grad_u / veclen(grad_u)[:,np.newaxis]
        # heat method step 3
        div_Xs = np.zeros(len(self._verts))
        for i1, i2, i3 in [(0, 1, 2), (1, 2, 0), (2, 0, 1)]: # for edge i2 --> i3 facing vertex i1
            vi1, vi2, vi3 = self._tris[:,i1], self._tris[:,i2], self._tris[:,i3]
            e1 = self._verts[vi2] - self._verts[vi1]
            e2 = self._verts[vi3] - self._verts[vi1]
            e_opp = self._verts[vi3] - self._verts[vi2]
            cot1 = 1 / np.tan(np.arccos( 
                (normalized(-e2) * normalized(-e_opp)).sum(axis=1)))
            cot2 = 1 / np.tan(np.arccos(
                (normalized(-e1) * normalized( e_opp)).sum(axis=1)))
            div_Xs += np.bincount(
                vi1.astype(int), 
        0.5 * (cot1 * (e1 * X).sum(axis=1) + cot2 * (e2 * X).sum(axis=1)), 
        minlength=len(self._verts))
        phi = self._factored_L(div_Xs).ravel()
        phi -= phi.min()
        return phi

In [140]:
def perpendicular_dist(start_point, grad_val1, grad_val2, interp_data_grad):
    
    step_size = 50
    step = 0
    a = start_point
    
    if grad_val1 < grad_val2:
        for i in range (1000):
            # calculate scalar field vector at point
            grad_vec = gp.compute_model_at(a, interp_data_grad)
            # break condition if surface was crossed
            if grad_vec[0][1] > grad_val2:
                break
            step += 1
            grad_vec =np.array([grad_vec[0][2],grad_vec[0][3],grad_vec[0][4]])
            # normalize to step size and set direction
            vec_len = np.linalg.norm((grad_vec), ord=1)
            grad_vec_norm = step_size*grad_vec/vec_len
            grad_vec_norm = np.reshape(grad_vec_norm, (1, 3))
            a = a+grad_vec_norm
            
    else:
        for i in range (1000):
            # calculate scalar field vector at point
            grad_vec = gp.compute_model_at(a, interp_data_grad)
            # break condition if surface was crossed
            if grad_vec[0][1] < grad_val2:
                break
            step += 1
            grad_vec =np.array([grad_vec[0][2],grad_vec[0][3],grad_vec[0][4]])
            # normalize to step size and set direction
            vec_len = np.linalg.norm((grad_vec), ord=1)
            grad_vec_norm = step_size*grad_vec/vec_len
            grad_vec_norm = np.reshape(grad_vec_norm, (1, 3))
            grad_vec_norm = grad_vec_norm*(-1)
            a = a+grad_vec_norm
    
    dist = step*step_size
    point = a
    return dist, point

def parallel_dist(vertices, simplices, point1, point2):
    compute_distance = GeodesicDistanceComputation(vertices, simplices)
    distance_of_each_vertex_to_vertex_0 = compute_distance(point1)
    dist = distance_of_each_vertex_to_vertex_0[point2]
    return dist

In [207]:
def get_distance(pointA, pointB, geo_data, interp_data_grad, a, res, gradA, vertices_pA, simplices_pA):
    
    # compute value of scalar field at given points, rounded to avoid singular matrices
    gradB = np.round(gp.compute_model_at(pointB, interp_data)[0][1], 2)

    # get mesh plane for second point
    vertices_pB, simplices_pB, normalsB, valuesB = measure.marching_cubes_lewiner(
            a,
            gradB,
            step_size=3,
            spacing=((geo_data.extent[1]/geo_data.resolution[0]),(geo_data.extent[3]/geo_data.resolution[1]),(geo_data.extent[5]/geo_data.resolution[2])))

    # select closest points in mesh
    closeA = cdist(pointA, vertices_pA).argmin()
    closeB = cdist(pointB, vertices_pB).argmin()
    
    # calculate perpendicular distance towards each triangular mesh respectively
    dist_perpA, pointA2 = perpendicular_dist(pointA, gradA, gradB, interp_data_grad)
    dist_perpB, pointB2 = perpendicular_dist(pointB, gradB, gradA, interp_data_grad)
    closeA2, distA2 = closest_node(pointA2, vertices_pB)
    closeB2, distA2 = closest_node(pointB2, vertices_pA)
    
    # calculate parallel distance
    dist_paraA = parallel_dist(vertices_pA, simplices_pA, closeA, closeB2)
    dist_paraB = parallel_dist(vertices_pB, simplices_pB, closeB, closeA2)
    
    # calculate full distance
    total_dist = 0.5*(dist_paraA+dist_paraB)+0.5*(dist_perpA+dist_perpB)
    
    return total_dist

In [208]:
def calculate_distance_set(point, points):
    result = np.zeros((len(points)))
    
    # do precalculations, mesh through basic point (only once)
    res = gp.get_resolution(geo_data)
    a = lith_block[1].reshape(res)
    gradA = np.round(gp.compute_model_at(pointA, interp_data)[0][1], 2)
    vertices_pA, simplices_pA, normalsA, valuesA = measure.marching_cubes_lewiner(
            a,
            gradA,
            step_size=3,
            spacing=((geo_data.extent[1]/geo_data.resolution[0]),(geo_data.extent[3]/geo_data.resolution[1]),(geo_data.extent[5]/geo_data.resolution[2])))
    
    for i in range (len(points)):
        result[i] = get_distance(point, points[[i]], geo_data, interp_data_grad, a, res, gradA, vertices_pA, simplices_pA)
    return result

In [209]:
# create domain data for certain lithology
domain_data = init_domain(lith_block, geo_data.grid, formation=3)

In [210]:
point = np.array([[domain_data.X[100], domain_data.Y[100], domain_data.Z[100]]])
n = 50
dist_points = np.zeros((n, 3))
for i in range (n):
    rand = np.random.randint(0,3750)
    dist_points[i][0] = domain_data.X[rand]
    dist_points[i][1] = domain_data.Y[rand]
    dist_points[i][2] = domain_data.Z[rand]    

This is where I ceck my speed with 50 points:

For following parameters i get the following results (approximately)
- step size mesh: 2 - only thing that works in general
- step_size vert_dist: 5 - 24s, 20 - 7s, 50 - 3.5s --- obviuosly great impact on speed
- got 1s by taking out one compute-model at in the vert. dist.
- removed closest node function - no improvement
- got another 1-2s by only calculating the surface for the basic point once

In [212]:
%timeit test = calculate_distance_set(point, dist_points)
print(test)

C:\Users\Jan\Anaconda3\lib\site-packages\theano\tensor\subtensor.py:2320: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[0][inputs[2:]] = inputs[1]


2.46 s ± 44.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
[2509.06876133 2776.41615869 2684.69641385  297.78886782 1302.17941446
  198.11045859  919.4939678   890.83521082  783.83693471 3128.35010784
 2836.62876444  512.04062596 3106.44396755 2140.62050759 1665.43673989
 2094.36866136  302.59688992 2140.62050759 2577.29476519 2177.58667226
  586.08494565 2165.62050759  284.3211291   986.42858548 2984.29377857
 1714.78349135 2145.67212326  214.85353406 1243.33917806 3226.93235724
 3121.07287577 1380.24092373 1439.95300374  931.48562213 2388.23223831
 1068.67521758 1954.01513622 3164.43185836  474.89860611 3124.19475683
 1134.26602262 1524.62012809 3209.43913919 1533.22876915 2333.47285639
 1777.40319591 1680.87350713 1327.26592683  737.06974869 2748.74417511]


In [201]:
%%time
compare = cdist(point, dist_points)
print(compare)

[[2378.31498012  450.16818425 2559.67198648 1305.6942708   673.91464267
  1438.9326131  1741.24346687  282.7833052   989.43664325  199.9027592
  1260.2245231  2342.07160407 2700.46683709 2240.27582237   90.25595961
  2779.16899212 1654.42131001  933.7578673   268.18030944  375.3394
  1514.40448882 2883.58209149 2675.47120044 2080.42257149 2011.47921738
  2360.83737945 2147.00609293  554.81618755  695.96856936  554.81618755
  2129.21016943 2903.12125153  764.10277044 2137.5913844  1978.65058364
  1535.89871769 1914.69283227 1138.17335855  882.87074948 1951.17280254
   693.81995064 2807.01551708  472.25898409 1298.08158158 1347.07616422
   447.40697076 1903.17505904  397.18341505 2119.46883095 1849.38419716]]
Wall time: 4 ms
